<a href="https://colab.research.google.com/github/JNicastri1/projetoML/blob/main/projeto_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/datasets/samayashar/fraud-detection-transactions-dataset/data

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_excel("synthetic_fraud_dataset.xlsx")

data.head()

,"Transaction_ID,User_ID,Transaction_Amount,Transaction_Type,Timestamp,Account_Balance,Device_Type,Location,Merchant_Category,IP_Address_Flag,Previous_Fraudulent_Activity,Daily_Transaction_Count,Avg_Transaction_Amount_7d,Failed_Transaction_Count_7d,Card_Type,Card_Age,Transaction_Distance,Authentication_Method,Risk_Score,Is_Weekend,Fraud_Label"
0,"TXN_33553,USER_1834,39.79,POS,2023-08-14 19:30..."
1,"TXN_9427,USER_7875,1.19,Bank Transfer,2023-06-..."
2,"TXN_199,USER_2734,28.96,Online,2023-06-20 15:2..."
3,"TXN_12447,USER_2617,254.32,ATM Withdrawal,2023..."
4,"TXN_39489,USER_2014,31.28,POS,2023-11-11 23:44..."
